In [66]:
import cbpro
import pandas as pd

from playsound import playsound

from api import Coinbase_API
from api import Coinbase_secret
from api import Coinbase_pass

api_key = Coinbase_API
api_secret = Coinbase_secret
api_pass = Coinbase_pass

from coinbase.wallet.client import Client

### WebsocketClient method

In [2]:
help(cbpro.WebsocketClient)

Help on class WebsocketClient in module cbpro.websocket_client:

class WebsocketClient(builtins.object)
 |  WebsocketClient(url='wss://ws-feed.pro.coinbase.com', products=None, message_type='subscribe', mongo_collection=None, should_print=True, auth=False, api_key='', api_secret='', api_passphrase='', channels=None)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, url='wss://ws-feed.pro.coinbase.com', products=None, message_type='subscribe', mongo_collection=None, should_print=True, auth=False, api_key='', api_secret='', api_passphrase='', channels=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  close(self)
 |  
 |  on_close(self)
 |  
 |  on_error(self, e, data=None)
 |  
 |  on_message(self, msg)
 |  
 |  on_open(self)
 |  
 |  start(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakr

### Retrieve ticker feed

In [1]:
import time, cbpro
class myWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):
        self.url = "wss://ws-feed.pro.coinbase.com/"
        self.products = ["ETH-USDT"]
        self.channels=["ticker"]
        self.message_count = 0
    def on_message(self, msg):
        self.message_count += 1
        if 'price' in msg and 'type' in msg:
            print ("Message type:", msg["type"],
                   "\t@ {:.3f}".format(float(msg["price"])))
    def on_close(self):
        print("Closing")

wsClient = myWebsocketClient()
wsClient.start()
print(wsClient.url, wsClient.products, wsClient.channels)
while (wsClient.message_count < 10):
    print ("\nmessage_count =", "{} \n".format(wsClient.message_count))
    time.sleep(1)
wsClient.close()

wss://ws-feed.pro.coinbase.com ['ETH-USDT'] ['ticker']

message_count = 0 

Message type: ticker 	@ 3282.130

message_count = 2 


message_count = 2 


message_count = 2 


message_count = 2 


message_count = 2 

Message type: ticker 	@ 3282.490
Message type: ticker 	@ 3282.490
Message type: ticker 	@ 3282.490
Message type: ticker 	@ 3282.470
Message type: ticker 	@ 3282.470
Message type: ticker 	@ 3282.140
Message type: ticker 	@ 3281.960

message_count = 9 

Message type: ticker 	@ 3281.940
Message type: ticker 	@ 3280.990
Closing


### Stream ticker feed

In [74]:
class TextWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):
        self.url = "wss://ws-feed.pro.coinbase.com/"
        self.products = ["ETH-USDT"]
        self.channels=["ticker"]
        self.message_count = 0
    def on_message(self, msg):
        self.message_count += 1
        msg_type= msg.get('type', None)
        if msg_type == 'ticker':
            time_val = msg.get('time',(' - '*27))
            price_val = msg.get('price', None)
            price_val = float(price_val) if price_val is not None else 'None'
            product_id = msg.get('product_id', None)
            print(price_val, end="\r") 
            
            if price_val > 3255:
                playsound('/Users/albertopuentes/Music/Music/Media.localized/Music/Unknown Artist/Unknown Album/Tolling_Bell.wav')
            
    def on_close(self):
        print(f'Websocket connection closing')

In [82]:
stream = TextWebsocketClient(products=['ETH-USD'],channels=['ticker'])
stream.start()

### Authenticated Client

In [8]:
from api import Coinbase_API
from api import Coinbase_secret
from api import Coinbase_pass

api_key = Coinbase_API
api_secret = Coinbase_secret
api_pass = Coinbase_pass

In [9]:
url = 'https://api.pro.coinbase.com'
client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass,
    api_url = url)

In [10]:
accounts = client.get_accounts()

In [11]:
for acc in accounts:
    currency = acc.get('currency')
    if currency == 'SHIB':
        acc_id = acc.get('id')

In [12]:
acc_history = client.get_account_history(acc_id)

In [14]:
import json

#for hist in acc_history:
    print(json.dumps(hist, indent=1))
    

IndentationError: unexpected indent (<ipython-input-14-7b1d23c4bee4>, line 4)

### Public Client

In [ ]:
c = cbpro.PublicClient()

data = pd.DataFrame(c.get_products())
data.tail().T

In [ ]:
ticker = c.get_product_ticker(product_id='ETH-USD')
ticker

In [ ]:
import requests

ticker = requests.get('https://api.pro.coinbase.com/products/ADA-USD/ticker').json()
ticker